Gen List

In [128]:
import pandas as pd
from utilsData import fuelDict

df = pd.read_excel('data/Refrence/2023_celt_report.xlsx', skiprows=13, index_col=None, sheet_name='2.1 Generator List')
df2 = df[['RESOURCE ID', 'RESOURCE NAME', 'GENERATOR TYPE', 'PRIM FUEL TYPE', 'NAMEPLATE\n(MW)']]
df2.rename(columns={"NAMEPLATE\n(MW)": "Nameplate Capacity (MW)", "PRIM FUEL TYPE": "Primary Fuel Type",
                     "RESOURCE ID": 'ID', "RESOURCE NAME": "Name"}, inplace=True)


fuels = df2['Primary Fuel Type'].map(fuelDict)
fuels = fuels.fillna('Other')
df2.insert(3, 'Fuel Type', fuels)

In [129]:
# ID_celt = df2[['ID']]
# ID_celt.dropna(inplace=True)

CSO

In [130]:
import pandas as pd
import numpy as np
from datetime import datetime

In [131]:
df = pd.read_excel('data/Refrence/fca_obligations.xlsx', skiprows=0, index_col=None, sheet_name='FCA 14')
df = df.drop(['Intermittent', 'DR Type', 'Active/ Passive', 'Interface ID', 'Interface Name', 'Dispatch Zone ID', 'Dispatch Zone Name',
              'Lead Participant ID', 'Lead Participant Name', 'De-list Bid Type', 'De-list Bid MW', 'Status', 'Capacity Zone ID', 'Capacity Zone Name', 'Load Zone'], axis=1)
new_columns = pd.to_datetime(df.columns[-12:]).strftime('%B')
df.columns = df.columns[:-12].tolist() + new_columns.tolist()
dftmp = df[df['Type'].isin(['Demand'])]
df = df[df['Type'].isin(['Generator'])]
df.reset_index(drop=True, inplace=True)

In [132]:
selected_columns = ['ID', 'Type', 'FCA Qual', 'June', 'July', 'August', 'September', 'October', 'November', 'December', 
                                            'January', 'February', 'March', 'April', 'May']

dfResult = df2.merge(df[selected_columns].set_index('ID'), on='ID')

new_column_order = ['ID', 'Name', 'GENERATOR TYPE' , 'Type', 'Fuel Type', 'Nameplate Capacity (MW)', 'FCA Qual', 
                    'June', 'July', 'August', 'September', 'October', 'November', 'December',
                    'January', 'February', 'March', 'April', 'May']
dfResult = dfResult[new_column_order]
# dfResult = pd.merge(df2, df[selected_columns], on='ID', how='outer')

In [133]:
# To add and fix the demand type
for col in dfResult.columns:
    if col not in dftmp.columns:
        dftmp[col] = np.nan
dftmp = dftmp[dfResult.columns]

dfResult = dfResult.append(dftmp, ignore_index=True)

dfResult.loc[dfResult['Type'] == 'Demand', 'Fuel Type'] = 'Demand'
dfResult.loc[dfResult['Type'] == 'Demand', 'Nameplate Capacity (MW)'] = dfResult.loc[dfResult['Type'] == 'Demand', 'FCA Qual']
# dfResult.loc[dfResult['Type'] == 'Demand', 'Name'] = dftmp.loc[dftmp['Type'] == 'Demand', 'Name'].values



/tmp/ipykernel_31511/3362554122.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfResult = dfResult.append(dftmp, ignore_index=True)


In [134]:
indices = (dfResult.iloc[:, 5:6].to_numpy() < dfResult.iloc[:, 6:].to_numpy()).any(axis=1)
tmp = dfResult.iloc[indices, 6:].max(axis=1) * 1.05
dfResult.iloc[indices, 5] = tmp.to_list()
(dfResult.iloc[:, 5:6].to_numpy() >= dfResult.iloc[:, 6:].to_numpy()).all(axis=1).all()

True

In [135]:
dfResult = dfResult.drop_duplicates(subset=['ID'], keep='first')
dfResult.to_csv('data/generation.csv', index=False)